In [ ]:
# =============================================================================
# NOTAS INICIALES: Este script simula un Júpiter Notebook o entorno similar,
# estructurando el flujo de trabajo para el preprocesamiento de los datos
# visuales (imágenes y metadata) del dataset.
# =============================================================================

# %% [markdown]
# # Proyecto: Medición de Estados Afectivos en Aulas Híbridas
# ## Módulo 1: Preprocesamiento y Extracción de Características Visuales
#
# Este módulo se centra en la ingesta, limpieza y preparación de los datos
# provenientes de las carpetas `images` y `metadata`, utilizando las etiquetas
# de la carpeta `labels` como Ground Truth (verdad fundamental).
#
# **Objetivos del Notebook:**
# 1.  Cargar y sincronizar los archivos de imagen (`.jpg`) con sus archivos de metadata (`.json`).
# 2.  Utilizar las coordenadas de detección facial de la metadata para realizar un recorte (crop) en las imágenes.
# 3.  Integrar las etiquetas humanas (inter-rater agreement) para crear el Ground Truth final.
# 4.  Generar un DataFrame unificado listo para el entrenamiento del modelo.

# %% [imports]
import os
import json
import pandas as pd
import cv2  # OpenCV para manipulación de imágenes
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from collections import defaultdict
from tqdm import tqdm

# %% [setup]
# =============================================================================
# 1. Configuración de Rutas y Constantes
# =============================================================================
# NOTA: Se asume que este script se ejecuta desde un directorio que contiene
# las carpetas 'images', 'labels' y 'metadata'.

BASE_DIR = os.getcwd() # Directorio actual
IMAGES_DIR = os.path.join(BASE_DIR, 'images')
METADATA_DIR = os.path.join(BASE_DIR, 'metadata')
LABELS_DIR = os.path.join(BASE_DIR, 'labels')

# Definiciones de etiquetas (adaptar según la Escala Likert o las categorías usadas)
# Ejemplo basado en el documento:
ATENTION_MAPPING = {
    '1': 'Baja', '2': 'Media', '3': 'Alta' # Ejemplo de una escala de 3 puntos
}
EMOTION_MAPPING = {
    '1': 'Neutral', '2': 'Felicidad', '3': 'Tristeza', '4': 'Sorpresa',
    '5': 'Ira', '6': 'Disgusto', '7': 'Miedo', '8': 'Desinterés' # Ejemplo de 8 categorías
}

# Clases objetivo del modelo (Ground Truth)
TARGET_ATTENTION_COL = 'attention_consensus'
TARGET_EMOTION_COL = 'emotion_consensus'

# %% [section]
# =============================================================================
# 2. Funciones de Carga y Preprocesamiento de Etiquetas Humanas (Ground Truth)
# =============================================================================

def load_and_merge_labels(labels_dir):
    """
    Carga todos los archivos JSON de la carpeta 'labels', los fusiona y
    calcula un consenso de etiquetas (Ground Truth).
    """
    all_labels = []
    label_files = glob(os.path.join(labels_dir, 'labeler_*.json'))
    # También incluimos la auto-etiquetación si existe (ej. self_labeling.json)
    self_label_file = os.path.join(labels_dir, 'self_labeling.json')
    if os.path.exists(self_label_file):
        label_files.append(self_label_file)

    print(f"Archivos de etiquetas encontrados: {len(label_files)}")

    for filepath in label_files:
        labeler_id = os.path.basename(filepath).split('.')[0]
        try:
            with open(filepath, 'r') as f:
                data = json.load(f)
                df_temp = pd.DataFrame(data)
                df_temp['labeler_id'] = labeler_id
                all_labels.append(df_temp)
        except Exception as e:
            print(f"Error al cargar {filepath}: {e}")

    if not all_labels:
        print("No se cargaron archivos de etiquetas.")
        return pd.DataFrame()

    df_raw = pd.concat(all_labels, ignore_index=True)

    # Convertir a numérico y manejar errores
    df_raw['attention'] = pd.to_numeric(df_raw['attention'], errors='coerce')
    df_raw['emotion'] = pd.to_numeric(df_raw['emotion'], errors='coerce')

    # Calcular el consenso: se utiliza la moda (el valor más votado)
    consensus_df = df_raw.groupby('datetime').agg(
        attention_consensus=('attention', lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan),
        emotion_consensus=('emotion', lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan),
        n_labels_attention=('attention', 'count'),
        n_labels_emotion=('emotion', 'count')
    ).reset_index()

    # Eliminar NaNs generados por filas vacías o datos inválidos
    consensus_df.dropna(subset=[TARGET_ATTENTION_COL, TARGET_EMOTION_COL], how='all', inplace=True)

    print(f"Etiquetas de consenso generadas para {len(consensus_df)} momentos.")
    return consensus_df

# %% [section]
# =============================================================================
# 3. Funciones de Carga y Preprocesamiento de Metadata
# =============================================================================

def load_and_flatten_metadata(metadata_dir):
    """
    Carga todos los archivos JSON de la carpeta 'metadata', extrae las
    características de bajo nivel y las consolida en un DataFrame.
    """
    metadata_list = []
    metadata_files = glob(os.path.join(metadata_dir, '*.json'))

    print(f"Archivos de metadata encontrados: {len(metadata_files)}")

    for filepath in tqdm(metadata_files, desc="Procesando metadata"):
        filename = os.path.basename(filepath)
        datetime_id = filename.split('.')[0] # Asumiendo que el nombre es la marca de tiempo

        try:
            with open(filepath, 'r') as f:
                data = json.load(f)
                row = {'datetime': datetime_id, 'metadata_file': filename}

                # 1. Extracción de Bounding Box y Características Demográficas
                face_data = data.get('person', {}).get('face', {})
                if not face_data:
                    continue # Saltar si no hay detección de rostro
                
                # Bounding Box
                bbox = face_data.get('bounding_box', {})
                row.update({f'bbox_{k}': v for k, v in bbox.items()})

                # Demográficos
                row['age'] = face_data.get('age')
                row['gender_name'] = face_data.get('gender', {}).get('gender_name')

                # 2. Extracción de Probabilidades de Emoción (Multimodal - Facial)
                emotion_probs = face_data.get('emotion', {}).get('probability_emotion', {})
                row.update({f'prob_emotion_{k}': v for k, v in emotion_probs.items()})
                row['dominant_emotion'] = face_data.get('emotion', {}).get('dominant_emotion')
                
                # 3. Extracción de Features de Postura (Multimodal - Postural)
                # Se pueden extraer características resumidas de los 'landmarks'
                # Por ejemplo, la visibilidad promedio de los landmarks del cuerpo
                landmarks = data.get('person', {}).get('landmarks', [])
                visibility = [l.get('visibility', 0) for l in landmarks]
                presence = [l.get('presence', 0) for l in landmarks]
                
                row['avg_landmark_visibility'] = np.mean(visibility) if visibility else 0
                row['avg_landmark_presence'] = np.mean(presence) if presence else 0

                metadata_list.append(row)

        except Exception as e:
            print(f"Error al procesar metadata en {filename}: {e}")

    df_metadata = pd.DataFrame(metadata_list)
    print(f"Metadata consolidada para {len(df_metadata)} registros.")
    return df_metadata

# %% [section]
# =============================================================================
# 4. Función de Preprocesamiento de Imágenes (Recorte Facial)
# =============================================================================

def crop_and_save_face(row, output_dir='processed_faces', margin_ratio=0.3):
    """
    Carga la imagen, utiliza el Bounding Box de la metadata y recorta el rostro,
    guardando el resultado en una nueva carpeta.
    """
    img_filename = f"{row['datetime']}.jpg"
    img_path = os.path.join(IMAGES_DIR, img_filename)
    output_path = os.path.join(output_dir, img_filename)

    # Crear directorio de salida si no existe
    os.makedirs(output_dir, exist_ok=True)

    if not os.path.exists(img_path):
        # print(f"Imagen no encontrada: {img_path}")
        return None # Devuelve None si la imagen no existe

    try:
        # Cargar imagen
        image = cv2.imread(img_path)
        if image is None:
            raise FileNotFoundError("Error al cargar la imagen con OpenCV")

        H, W = image.shape[:2]

        # Extraer coordenadas del Bounding Box
        x0 = int(row['bbox_x0'])
        y0 = int(row['bbox_y0'])
        x1 = int(row['bbox_x1'])
        y1 = int(row['bbox_y1'])

        # Calcular el ancho y alto originales del BBox
        w_orig = x1 - x0
        h_orig = y1 - y0

        # Añadir un margen (30% del tamaño original) para contexto
        margin_w = int(w_orig * margin_ratio)
        margin_h = int(h_orig * margin_ratio)

        # Recalcular las coordenadas con margen, asegurando que no se salgan de la imagen
        x_min = max(0, x0 - margin_w)
        y_min = max(0, y0 - margin_h)
        x_max = min(W, x1 + margin_w)
        y_max = min(H, y1 + margin_h)

        # Recortar la región del rostro con margen
        cropped_face = image[y_min:y_max, x_min:x_max]

        # Guardar la imagen recortada
        cv2.imwrite(output_path, cropped_face)

        return output_path

    except Exception as e:
        print(f"Error procesando imagen {img_filename}: {e}")
        return None

def visualize_sample_image(df, n_samples=3, processed_dir='processed_faces'):
    """
    Muestra una comparación visual de la imagen original vs. la imagen recortada.
    """
    if df.empty:
        print("DataFrame vacío, no se puede visualizar.")
        return
    
    sample = df.sample(min(n_samples, len(df)))
    
    fig, axes = plt.subplots(n_samples, 2, figsize=(10, 5 * n_samples))
    
    if n_samples == 1: # Para manejar el caso de una sola muestra
        axes = np.expand_dims(axes, axis=0)

    for i, (_, row) in enumerate(sample.iterrows()):
        datetime_id = row['datetime']
        original_img_path = os.path.join(IMAGES_DIR, f"{datetime_id}.jpg")
        processed_img_path = os.path.join(processed_dir, f"{datetime_id}.jpg")
        
        # Imagen Original
        if os.path.exists(original_img_path):
            img_orig = cv2.cvtColor(cv2.imread(original_img_path), cv2.COLOR_BGR2RGB)
            axes[i, 0].imshow(img_orig)
            axes[i, 0].set_title(f"Original: {datetime_id}")
            axes[i, 0].axis('off')
        
        # Imagen Recortada
        if os.path.exists(processed_img_path):
            img_proc = cv2.cvtColor(cv2.imread(processed_img_path), cv2.COLOR_BGR2RGB)
            axes[i, 1].imshow(img_proc)
            attention = ATENTION_MAPPING.get(str(int(row[TARGET_ATTENTION_COL])), 'N/A')
            emotion = EMOTION_MAPPING.get(str(int(row[TARGET_EMOTION_COL])), 'N/A')
            axes[i, 1].set_title(f"Recorte (GT: Att={attention}, Emo={emotion})")
            axes[i, 1].axis('off')
            
    plt.tight_layout()
    plt.show()

# %% [execution]
# =============================================================================
# 5. Pipeline Principal de Ejecución
# =============================================================================
if __name__ == "__main__":
    print("--- 1. Carga y Consenso de Etiquetas Humanas (Ground Truth) ---")
    df_labels = load_and_merge_labels(LABELS_DIR)

    if df_labels.empty:
        print("Proceso terminado: No se pudo generar el DataFrame de etiquetas.")
        exit()

    print("\n--- 2. Carga y Consolidación de Metadata (Características) ---")
    df_metadata = load_and_flatten_metadata(METADATA_DIR)

    if df_metadata.empty:
        print("Proceso terminado: No se pudo generar el DataFrame de metadata.")
        exit()

    print("\n--- 3. Fusión de Datos (Labels + Metadata) ---")
    # Realizar el merge por el identificador de tiempo 'datetime'
    df_final = pd.merge(df_metadata, df_labels, on='datetime', how='inner')

    print(f"DataFrame Final Unificado (Registros listos para entrenamiento): {len(df_final)}")
    print(df_final.head())

    # --- 4. Preprocesamiento de Imágenes ---
    PROCESSED_FACES_DIR = os.path.join(BASE_DIR, 'processed_faces')
    print(f"\n--- 4. Procesamiento Visual (Recorte Facial) en: {PROCESSED_FACES_DIR} ---")
    
    # Aplicar la función de recorte a cada fila del DataFrame final
    tqdm.pandas(desc="Recortando y guardando rostros")
    df_final['processed_img_path'] = df_final.progress_apply(crop_and_save_face, axis=1, output_dir=PROCESSED_FACES_DIR)

    # Limpiar registros donde la imagen no pudo ser procesada o no existe
    df_final.dropna(subset=['processed_img_path'], inplace=True)
    
    print(f"\nDataFrame Final después del procesamiento de imágenes: {len(df_final)}")
    
    # --- 5. Exportación del DataFrame Final ---
    FINAL_CSV_PATH = os.path.join(BASE_DIR, 'processed_multimodal_data.csv')
    df_final.to_csv(FINAL_CSV_PATH, index=False)
    print(f"\nDatos consolidados exportados a: {FINAL_CSV_PATH}")

    # --- 6. Visualización de Muestra ---
    print("\n--- 6. Visualización de Muestras (Original vs. Recortada) ---")
    # Requiere tener las librerías cv2 y matplotlib instaladas.
    # visualize_sample_image(df_final, n_samples=3, processed_dir=PROCESSED_FACES_DIR)

    print("\nProceso de preprocesamiento visual completado.")
# %% [end]